In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
from utils import calculate_returns, compute_atr, compute_macd
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

master_ticker_data = sp500.copy()
master_ticker_data.rename(columns={"Symbol":"ticker"}, inplace = True)
master_ticker_data['ticker'] = master_ticker_data['ticker'].str.replace('.', '-')

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-12-31'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['GEV', 'SW', 'SOLV']: YFChartError("%ticker%: Data doesn't exist for startDate = 1388638800, endDate = 1703998800")


Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-01-02 A        36.805557   40.207439   40.844063   40.164520   40.844063   
           AAL      23.907923   25.360001   25.820000   25.059999   25.070000   
           AAPL     17.253262   19.754642   19.893929   19.715000   19.845715   
           ABBV     33.557144   51.980000   52.330002   51.520000   52.119999   
           ABT      31.049803   38.230000   38.400002   38.000000   38.090000   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.729843  114.360001  114.680000  113.930000  114.089996   
           YUM     129.373611  130.660004  131.250000  130.210007  130.410004   
           ZBH     121.202194  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     195.897491  197.369995  198.009995  196.250000  196.679993   

Price                   volume  
date       ticker               
2014-01-02 A         2678848.0  
           AAL       8997900.0  
           AAPL    234684800.0  
           ABBV      4569100.0  
           ABT       4967500.0  
...                        ...  
2023-12-29 XYL        698900.0  
           YUM       1196800.0  
           ZBH        849600.0  
           ZBRA       251300.0  
           ZTS       1007200.0  

[1225200 rows x 6 columns]

In [2]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

df['sharpe_ratio'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.sharpe_ratio(close=x))

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-01-02 A        36.805557   40.207439   40.844063   40.164520   40.844063   
           AAL      23.907923   25.360001   25.820000   25.059999   25.070000   
           AAPL     17.253262   19.754642   19.893929   19.715000   19.845715   
           ABBV     33.557144   51.980000   52.330002   51.520000   52.119999   
           ABT      31.049803   38.230000   38.400002   38.000000   38.090000   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.729843  114.360001  114.680000  113.930000  114.089996   
           YUM     129.373611  130.660004  131.250000  130.210007  130.410004   
           ZBH     121.202194  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     195.897491  197.369995  198.009995  196.250000  196.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-01-02 A         2678848.0         -0.004046        NaN       NaN   
           AAL       8997900.0         -0.000424        NaN       NaN   
           AAPL    234684800.0         -0.007529        NaN       NaN   
           ABBV      4569100.0         -0.074767        NaN       NaN   
           ABT       4967500.0         -0.016078        NaN       NaN   
...                        ...               ...        ...       ...   
2023-12-29 XYL        698900.0          0.000018  75.881258  4.647615   
           YUM       1196800.0          0.000007  59.338363  4.814819   
           ZBH        849600.0          0.000044  68.767234  4.749500   
           ZBRA       251300.0          0.000073  67.914673  5.415905   
           ZTS       1007200.0          0.000034  66.791325  5.179002   

Price                bb_mid   bb_high  sharpe_ratio       atr      macd  \
date       ticker                                                         
2014-01-02 A            NaN       NaN      0.638107       NaN       NaN   
           AAL          NaN       NaN      0.137144       NaN       NaN   
           AAPL         NaN       NaN      0.992655       NaN       NaN   
           ABBV         NaN       NaN      0.697839       NaN       NaN   
           ABT          NaN       NaN      0.652657       NaN       NaN   
...                     ...       ...           ...       ...       ...   
2023-12-29 XYL     4.702070  4.756525      0.632794 -0.071904  2.215117   
           YUM     4.852975  4.891132      0.565898  0.100551  0.893188   
           ZBH     4.780406  4.811312      0.278126 -0.907728  1.063465   
           ZBRA    5.546306  5.676707      0.614979  0.079474  1.761343   
           ZTS     5.251999  5.324997      0.861607  0.649087  2.432946   

Price              dollar_volume  
date       ticker                 
2014-01-02 A           98.596493  
           AAL        215.121098  
           AAPL      4049.078240  
           ABBV       153.325947  
           ABT        154.239895  
...                          ...  
2023-12-29 XYL         79.485787  
           YUM        154.834338  
           ZBH        102.973384  
           ZBRA        68.687826  
           ZTS        197.307953  

[1225200 rows x 15 columns]

In [3]:
data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-02-07 A        38.907421   42.503578   42.539341   41.580830   41.816879   
           AAL      33.627583   35.669998   35.700001   34.650002   34.810001   
           AAPL     16.306919   18.559999   18.676071   18.477858   18.620714   
           ABBV     31.812323   48.889999   49.029999   47.660000   48.000000   
           ABT      30.365843   37.180000   37.209999   36.650002   36.770000   
...                       ...         ...         ...         ...         ...   
2023-12-29 XYL     113.729843  114.360001  114.680000  113.930000  114.089996   
           YUM     129.373611  130.660004  131.250000  130.210007  130.410004   
           ZBH     121.202194  121.699997  122.400002  121.239998  121.459999   
           ZBRA    273.329987  273.329987  276.309998  272.769989  274.730011   
           ZTS     195.897491  197.369995  198.009995  196.250000  196.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-02-07 A         2749586.0         -0.001749  56.320270  3.636126   
           AAL      15404700.0         -0.000016  83.448614  3.295002   
           AAPL    370280400.0         -0.006744  44.058852  2.786732   
           ABBV      6979500.0         -0.064962  46.153403  3.442998   
           ABT      12028000.0         -0.014031  46.146427  3.383648   
...                        ...               ...        ...       ...   
2023-12-29 XYL        698900.0          0.000018  75.881258  4.647615   
           YUM       1196800.0          0.000007  59.338363  4.814819   
           ZBH        849600.0          0.000044  68.767234  4.749500   
           ZBRA       251300.0          0.000073  67.914673  5.415905   
           ZTS       1007200.0          0.000034  66.791325  5.179002   

Price                bb_mid  ...  sharpe_ratio       atr      macd  \
date       ticker            ...                                     
2014-02-07 A       3.678775  ...      0.638107 -0.813500 -0.263342   
           AAL     3.427001  ...      0.137144  0.077450  2.813629   
           AAPL    2.862375  ...      0.992655 -0.929876 -0.511188   
           ABBV    3.489600  ...      0.697839 -0.934993 -0.613570   
           ABT     3.455548  ...      0.652657 -0.874355 -0.931537   
...                     ...  ...           ...       ...       ...   
2023-12-29 XYL     4.702070  ...      0.632794 -0.071904  2.215117   
           YUM     4.852975  ...      0.565898  0.100551  0.893188   
           ZBH     4.780406  ...      0.278126 -0.907728  1.063465   
           ZBRA    5.546306  ...      0.614979  0.079474  1.761343   
           ZTS     5.251999  ...      0.861607  0.649087  2.432946   

Price              dollar_volume  return_1m  return_2m  return_3m  return_6m  \
date       ticker                                                              
2014-02-07 A          106.979300   0.019386   0.017178   0.009198  -0.000252   
           AAL        518.022821   0.029140   0.027290   0.015614   0.009015   
           AAPL      6038.132526   0.013991   0.009901   0.009090   0.007530   
           ABBV       222.034106   0.020030   0.012079   0.006492   0.001991   
           ABT        365.240357   0.014461   0.011352   0.008480   0.002944   
...                          ...        ...        ...        ...        ...   
2023-12-29 XYL         79.485787   0.000350  -0.000087   0.002166   0.007004   
           YUM        154.834338   0.001073  -0.000688   0.001023   0.002082   
           ZBH        102.973384   0.000576   0.002350   0.000768   0.002680   
           ZBRA        68.687826  -0.007336  -0.004470  -0.002632   0.007331   
           ZTS        197.307953   0.001065   0.001193   0.003178   0.002333   

Price              return_9m  return_12m  
d

#### Download Fama-French Factors and Calculate Rolling Factor Betas.

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.

In [4]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2014-02-28 A       0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010114
           AAL     0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010121
           AAPL    0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.002710
           ABBV    0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.000000
           ABT     0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.000251
...                   ...     ...     ...     ...     ...        ...
2023-11-30 XYL     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.015749
           YUM     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.000239
           ZBH     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.020980
           ZBRA    0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.001521
           ZTS     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.005006

[40903 rows x 6 columns]

In [5]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2014-02-28 A       0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010114
           AAL     0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.010121
           AAPL    0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.002710
           ABBV    0.0465  0.0014 -0.0031 -0.0023 -0.0045   0.000000
           ABT     0.0465  0.0014 -0.0031 -0.0023 -0.0045  -0.000251
...                   ...     ...     ...     ...     ...        ...
2023-11-30 XYL     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.015749
           YUM     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.000239
           ZBH     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.020980
           ZBRA    0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.001521
           ZTS     0.0884 -0.0011  0.0161 -0.0390 -0.0104   0.005006

[40888 rows x 6 columns]

Calculate Rolling Factor Betas.

In [6]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2014-02-28 A            NaN       NaN       NaN       NaN       NaN
           AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-11-30 XYL     0.080510 -0.257672  0.115558 -0.000148 -0.262058
           YUM     0.062703 -0.158359  0.140724 -0.001534 -0.157107
           ZBH     0.142969 -0.244230  0.096570 -0.174764 -0.101753
           ZBRA    0.095924 -0.112022  0.187823 -0.037199 -0.230023
           ZTS     0.052170 -0.173515  0.039500  0.021199 -0.035151

[40888 rows x 5 columns]

Join the rolling factors data to the main features dataframe

In [7]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1211822 entries, (Timestamp('2014-02-07 00:00:00'), 'A') to (Timestamp('2023-12-29 00:00:00'), 'ZTS')
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   close             1211822 non-null  float64
 1   high              1211822 non-null  float64
 2   low               1211822 non-null  float64
 3   open              1211822 non-null  float64
 4   volume            1211822 non-null  float64
 5   garman_klass_vol  1211822 non-null  float64
 6   rsi               1211822 non-null  float64
 7   bb_low            1211822 non-null  float64
 8   bb_mid            1211822 non-null  float64
 9   bb_high           1211822 non-null  float64
 10  sharpe_ratio      1211822 non-null  float64
 11  atr               1211822 non-null  float64
 12  macd              1211822 non-null  float64
 13  dollar_volume     1211822 non-null  float64
 14  return_1m         1211822 

In [8]:
data.head()

close       high        low       open       volume  \
date       ticker                                                            
2014-02-07 A       42.503578  42.539341  41.580830  41.816879    2749586.0   
           AAL     35.669998  35.700001  34.650002  34.810001   15404700.0   
           AAPL    18.559999  18.676071  18.477858  18.620714  370280400.0   
           ABBV    48.889999  49.029999  47.660000  48.000000    6979500.0   
           ABT     37.180000  37.209999  36.650002  36.770000   12028000.0   

                   garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2014-02-07 A              -0.001749  56.320270  3.636126  3.678775  3.721424   
           AAL            -0.000016  83.448614  3.295002  3.427001  3.558999   
           AAPL           -0.006744  44.058852  2.786732  2.862375  2.938017   
           ABBV           -0.064962  46.153403  3.442998  3.489600  3.536202   
           ABT            -0.014031  46.146427  3.383648  3.455548  3.527449   

                   ...  return_2m  return_3m  return_6m  return_9m  \
date       ticker  ...                                               
2014-02-07 A       ...   0.017178   0.009198  -0.000252   0.002117   
           AAL     ...   0.027290   0.015614   0.009015   0.018744   
           AAPL    ...   0.009901   0.009090   0.007530  -0.005722   
           ABBV    ...   0.012079   0.006492   0.001991   0.004795   
           ABT     ...   0.011352   0.008480   0.002944   0.002665   

                   return_12m    Mkt-RF       SMB       HML       RMW  \
date       ticker                                                       
2014-02-07 A        -0.002089 -0.018337 -0.034140  0.027267  0.015874   
           AAL       0.011220  0.002699 -0.284824  0.101768 -0.110187   
           AAPL     -0.004447  0.054182  0.006107 -0.033355  0.143762   
           ABBV     -0.000323  0.058885  0.079770 -0.099024  0.210362   
           ABT      -0.001969 -0.025794  0.060281 -0.029739  0.135113   

                        CMA  
date       ticker            
2014-02-07 A       0.027903  
           AAL    -0.017601  
           AAPL    0.194969  
           ABBV   -0.056104  
           ABT    -0.039637  

[5 rows x 25 columns]

### Save price data

In [9]:
data.reset_index(inplace = True)

In [10]:
data.to_parquet("data/price_data.gzip", compression='gzip', index = False)
data.to_csv("data/price_data.csv", index = False)

In [11]:
data.head()

,date,ticker,close,high,low,open,volume,garman_klass_vol,rsi,bb_low,...,return_2m,return_3m,return_6m,return_9m,return_12m,Mkt-RF,SMB,HML,RMW,CMA
0,2014-02-07,A,42.503578,42.539341,41.580830,41.816879,2749586.0,-0.001749,56.320270,3.636126,...,0.017178,0.009198,-0.000252,0.002117,-0.002089,-0.018337,-0.034140,0.027267,0.015874,0.027903
1,2014-02-07,AAL,35.669998,35.700001,34.650002,34.810001,15404700.0,-0.000016,83.448614,3.295002,...,0.027290,0.015614,0.009015,0.018744,0.011220,0.002699,-0.284824,0.101768,-0.110187,-0.017601
2,2014-02-07,AAPL,18.559999,18.676071,18.477858,18.620714,370280400.0,-0.006744,44.058852,2.786732,...,0.009901,0.009090,0.007530,-0.005722,-0.004447,0.054182,0.006107,-0.033355,0.143762,0.194969
3,2014-02-07,ABBV,48.889999,49.029999,47.660000,48.000000,6979500.0,-0.064962,46.153403,3.442998,...,0.012079,0.006492,0.001991,0.004795,-0.000323,0.058885,0.079770,-0.099024,0.210362,-0.056104
4,2014-02-07,ABT,37.180000,37.209999,36.650002,36.770000,12028000.0,-0.014031,46.146427,3.383648,...,0.011352,0.008480,0.002944,0.002665,-0.001969,-0.025794,0.060281,-0.029739,0.135113,-0.039637


### Save data to sql db

In [12]:
import sqlite3
conn = sqlite3.connect('data/database.db')
c = conn.cursor()

In [13]:
data.to_sql('price_data', conn, if_exists='append', index = False)
master_ticker_data.to_sql('master_ticker', conn, if_exists='append')

503

### TODO: Run time series K Means clustering - DTW